In [ ]:
# !python.exe -m pip install --upgrade pip

In [3]:
!conda clean --all
!conda update --all

Retrieving notices: ...working... done
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - pylibcugraph=24.6.1
  - cudatoolkit=12.5

Current channels:

  - https://conda.anaconda.org/rapidsai/win-64
  - https://conda.anaconda.org/rapidsai/noarch
  - https://conda.anaconda.org/nvidia/win-64
  - https://conda.anaconda.org/nvidia/noarch
  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/pytorch/win-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch
  - https://conda.anaconda.org/anaconda/win-64
  - https://conda.anaconda.org/anaconda/noarch
  - https://conda.anaconda.org/numba/win-64
  - https://conda.anaconda.org/numba/noarch

To search

In [ ]:
import pandas as pd
import networkx as nx

url = "https://data.rapids.ai/cugraph/datasets/cit-Patents.csv"
df = pd.read_csv(url, sep=" ", names=["src", "dst"], dtype="int32")
G = nx.from_pandas_edgelist(df, source="src", target="dst")

%time result = nx.betweenness_centrality(G, k=10)

In [1]:
import sys
import subprocess

def verificar_versoes():
    """Verifica e exibe as versões do Python e CUDA instaladas no sistema."""

    # Versão do Python
    python_version = sys.version.split()[0]
    print(f"Versão do Python: {python_version}")

    # Versão do CUDA (usando nvcc)
    try:
        cuda_version = subprocess.check_output(["nvcc", "--version"])
        cuda_version = cuda_version.decode("utf-8").strip().split("\n")[-1]
        print(f"Versão   do CUDA: {cuda_version}")
    except FileNotFoundError:
        print("nvcc não encontrado. CUDA pode não estar instalado ou configurado corretamente.")

# Chamando a função para verificar as versões
verificar_versoes()

Versão do Python: 3.11.2
Versão   do CUDA: Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
%pip install --extra-index-url https://pypi.nvidia.com/pylibcugraph-cu12 pylibcugraph_cu12-24.6.1-cp311-cp311-manylinux_2_28_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com/pylibcugraph-cu12
Note: you may need to restart the kernel to use updated packages.


ERROR: pylibcugraph_cu12-24.6.1-cp311-cp311-manylinux_2_28_x86_64.whl is not a supported wheel on this platform.


In [5]:
!conda config --add channels conda-forge
!conda config --add channels rapidsai
!conda config --add channels nvidia
# %conda install -c rapidsai -c nvidia -c conda-forge pylibcugraph=24.6.1 cudatoolkit=12.5
%conda install -c rapidsai -c nvidia -c conda-forge pylibcugraph

Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - pylibcugraph

Current channels:

  - https://conda.anaconda.org/rapidsai/win-64
  - https://conda.anaconda.org/rapidsai/noarch
  - https://conda.anaconda.org/nvidia/win-64
  - https://conda.anaconda.org/nvidia/noarch
  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/pytorch/win-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch
  - https://conda.anaconda.org/anaconda/win-64
  - https://conda.anaconda.org/anaconda/noarch
  - https://conda.anaconda.org/numba/win-64
  - https://conda.anaconda.org/numba/noarch

To search for alternate channels that